## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-04-10-50-56 +0000,bbc,Who's in charge of Venezuela and what happens ...,https://www.bbc.com/news/articles/crmlz7r0zrxo...
1,2026-01-04-10-48-33 +0000,wapo,Live updates: Maduro held in New York as Trump...,https://www.washingtonpost.com/world/2026/01/0...
2,2026-01-04-10-48-33 +0000,wapo,As former Venezuelan leader Nicolás Maduro is ...,https://www.washingtonpost.com
3,2026-01-04-10-41-04 +0000,bbc,Starmer won't be drawn on whether US strikes o...,https://www.bbc.com/news/articles/cy4qgvwxp08o...
4,2026-01-04-10-37-39 +0000,nyt,Live Updates: Questions Mount for U.S. and Ven...,https://www.nytimes.com/live/2026/01/04/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,maduro,70
1,venezuela,58
10,trump,42
30,capture,25
17,nicol,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2026-01-04-10-48-33 +0000,wapo,Live updates: Maduro held in New York as Trump...,https://www.washingtonpost.com/world/2026/01/0...,219
184,2026-01-03-13-03-15 +0000,cbc,Trump says U.S. will 'run' Venezuela after it ...,https://www.cbc.ca/news/world/trump-maduro-ven...,215
53,2026-01-04-03-28-50 +0000,missionlocal,‘Latin America isn’t yours’: S.F. protesters c...,https://missionlocal.org/2026/01/sf-venezuela-...,207
153,2026-01-03-18-17-46 +0000,nypost,Trump brushes off concerns about legality of U...,https://nypost.com/2026/01/03/world-news/trump...,201
150,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. will 'run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...,199


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,219,2026-01-04-10-48-33 +0000,wapo,Live updates: Maduro held in New York as Trump...,https://www.washingtonpost.com/world/2026/01/0...
90,66,2026-01-04-00-10-00 +0000,nypost,Ousted Venezuelan dictator Nicolás Maduro arri...,https://nypost.com/2026/01/03/us-news/ousted-v...
25,39,2026-01-04-09-02-50 +0000,nypost,Heroic father saved trapped teens from Swiss s...,https://nypost.com/2026/01/04/world-news/swiss...
21,34,2026-01-04-10-00-00 +0000,wsj,There are 35 Senate seats up for election in 2...,https://www.wsj.com/politics/elections/democra...
54,30,2026-01-04-03-21-16 +0000,bbc,'A long road ahead': Venezuelans react to Madu...,https://www.bbc.com/news/articles/c86vq753nwpo...
148,29,2026-01-03-19-04-15 +0000,nypost,Inside Operation Absolute Resolve: How US forc...,https://nypost.com/2026/01/03/world-news/how-u...
42,27,2026-01-04-05-18-30 +0000,nypost,North Korea launches ballistic missiles toward...,https://nypost.com/2026/01/04/world-news/north...
157,24,2026-01-03-17-50-54 +0000,nypost,"Venezuelan dictator appears blindfolded, handc...",https://nypost.com/2026/01/03/world-news/venez...
31,23,2026-01-04-07-21-12 +0000,bbc,"'I'll be PM this time next year,' Starmer tell...",https://www.bbc.com/news/articles/clygv1ngynjo...
37,22,2026-01-04-06-02-11 +0000,bbc,'Deeply shocked': World leaders react to US at...,https://www.bbc.com/news/articles/czx1rpxzyx9o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
